In [20]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

In [22]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='file:///c:/Users/khadk/Desktop/MLOps/training/experimental_tracking/mlruns/1', creation_time=1729173508282, experiment_id='1', last_update_time=1729173508282, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [23]:
#@ code reconstruction:
def read_dataframe(filename):
    df=pd.read_parquet(filename)

    df['duration']=df.lpep_dropoff_datetime-df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda td: td.total_seconds()/60)

    df[((df.duration >= 1) & (df.duration <=60))]
    categorical=['PULocationID', 'DOLocationID']

    df[categorical]=df[categorical].astype(str)
    
    return df

In [24]:
df_train=read_dataframe('C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-01.parquet')
df_val=read_dataframe('C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-02.parquet')                  

In [25]:
df_train['PO_DO']=df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PO_DO']=df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [26]:
categorical=['PO_DO']#'PULocationID', 'DOLocationID']
numerical=['trip_distance']

dv=DictVectorizer()

train_dicts=df_train[categorical + numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)

val_dicts=df_val[categorical + numerical].to_dict(orient='records')
X_val=dv.transform(val_dicts)

In [27]:
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

In [28]:
lr=LinearRegression()
lr.fit(X_train, y_train)

y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

78.80821659744552

In [29]:
import os 

os.makedirs('Models', exist_ok=True)

with open('Models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr),f_out)

In [30]:
with mlflow.start_run():
    
    mlflow.set_tag('developer', 'king')
    mlflow.log_param('train-data-path', 'C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-01.parquet')
    mlflow.log_param('valid-data-path', 'C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-02.parquet')
    alpha=0.001 
    mlflow.log_param('alpha', alpha)
    lr=Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred=lr.predict(X_val)

    from sklearn.metrics import mean_squared_error

    rmse=mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)